In [27]:
############################################################
"""
    Importing initial Notebooks and Libraries
"""
############################################################

# Jupyter notebook imports
%run twitter_sentiment.ipynb

# Library imports
import os
import pandas as pd
from datetime import date, timedelta
import alpaca_trade_api as tradeapi
from talib import RSI, OBV

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

Twitter Authentication Verified


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Kris/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [28]:
############################################################
"""
    API Authentications for Alpaca
"""
############################################################
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [29]:
############################################################
"""
    Function scrapes Wikipedia and pulls stock data for every ticker symbol on the S&P500
"""
############################################################

# scrapes the wikipedia page relating to the S&P 500 and returns a list of DataFrame objects
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# Since we are only interested in the current list of stocks in the S&P 500, we only need the DataFrame object at index 0
sp500_list_df = table[0]
sp500_tickers = sp500_list_df[["Symbol", "Security"]]

############################################################
"""
    Function that fetches the Twitter Sentiment score
"""
############################################################

def fetch_twitter_sentiment(ticker, search_word):
    score = compound_twitter_sentiment(ticker, search_word)
    return score

############################################################
"""
    Stock Screener for Day Trading
    Start Main Function
"""
############################################################

# Initialize the dataframe and set the columns
sp500_ticker_data = []
column_names = ['time', 'open', 'low', 'close', 'volume', 'rsi', 'obv', 'volatility', 'twitter_sentiment', 'reddit_sentiment']
sp500_ticker_data = pd.DataFrame(columns = column_names)

# for loop iterates through all tickers in S&P 500
for tickers in range(len(sp500_tickers["Symbol"])):
    
    # set the current ticker symbol then increases x by 1 
    company = sp500_tickers.loc[[tickers][0]]
    ticker = company.Symbol
    security = company.Security
    
    # Set timeframe to 1 Day to capture daily Volume total
    timeframe = "1D"
    
    # Set the limit of bars to just the previous time period
    limit = 1

    # Set end date to now for latest data (if run before market open to fetch yesterdays close)
    end_date = pd.Timestamp.now(tz="America/New_York").isoformat()
    
    # Initial pass of ticker symbols
    # fetch last daily barset object for ticker and put raw data into dataframe
    bar_set = api.get_barset(
        ticker,
        timeframe,
        limit,
        end=end_date
        )[ticker]._raw
    initial_check = pd.DataFrame(data=bar_set)
    
    ############################################################
    """
    # Since Day traders generally look for stocks that have at least 1 million shares traded daily, 
    # this checks to see if the daily volume for current ticker >= 1Million.
    # This also checks the last closing price of a ticker to see if it is above $100.
    # If ticker volume < 1M or it's last closing price was above $100, 
    # we stop this iteration and continue to the next iteration of the loop.
    """
    ############################################################
        
    if initial_check.loc[0, 'v'] <= 1000000 or initial_check.loc[0, 'c'] > 100:
        continue
    else:
        # If the ticker passes both initial checks, it continues to calculating the additional indicators
        # Set timeframe to 15 Minutes
        timeframe = "15Min"
    
        #fetch ticker data 
        ticker_data = api.get_barset(
            ticker,
            timeframe,
            limit,
            end=end_date
            )
    
        #ticker_data inherits dict and bars inherits list. You can iterate over them accordingly
        for symbols in ticker_data:
            bars = ticker_data[symbols]
            for bar in bars:
                sp500_ticker_data.loc[symbols, 'time'] = bar.t
                sp500_ticker_data.loc[symbols, 'open'] = bar.o
                sp500_ticker_data.loc[symbols, 'low'] = bar.l
                sp500_ticker_data.loc[symbols, 'close'] = bar.c
                sp500_ticker_data.loc[symbols, 'volume'] = bar.v
        
        
        ############################################################
        """ 
            Relative strength suggests continued outperformance while relative weakness suggests continued underperformance.
            High RSI (usually above 70) may indicate a stock is overbought, therefore it is a sell signal. 
            Low RSI (usually below 30) indicates stock is oversold, which means a buy signal. 
        """
        ############################################################
        
        #Fetch RSI score
        #rsi_score = RSI(sp500_ticker_data.loc[tickers, 'close'], timeperiod=1)
        #sp500_ticker_data.loc[tickers, 'rsi'] = rsi_score
        
        ############################################################
        """ 
            On-balance volume (OBV) is a technical trading momentum indicator that 
            uses volume flow to predict changes in stock price.
            
            When both price and OBV are making higher peaks and higher troughs, 
            the upward trend is likely to continue.
        """
        ############################################################
        
        #Fetch On Balance Volume(OBV)
        #obv_score = OBV(sp500_ticker_data.loc[tickers, 'close'], sp500_ticker_data.loc[tickers, 'volume'])
        #sp500_ticker_data.loc[tickers, 'obv'] = obv_score
        
        ############################################################
        """ 
            Volatility
        """
        ############################################################
        
        # Fetch previos day data and calculate .std() ??
        #fetch Average True Range(ATR) volatility indicator
        #
        #atr_score = ATR(high, low, close, timeperiod=14)
        #sp500_ticker_data.loc[tickers, 'atr'] = atr_score
        
        ############################################################
        """ 
            Twitter and Reddit Compound Sentiment
        """
        ############################################################
        
        #fetches the Compound Sentiment score for the last 15 minutes of Tweets on Twitter
        #twitter_comp_sentiment_score = fetch_twitter_sentiment(ticker, security)
        #sp500_ticker_data.loc[tickers, 'twitter_sentiment'] = twitter_comp_sentiment_score
        
        #fetch reddit sentiment
        #reddit_sentiment = fetch_reddit_sentiment(ticker)
        #sp500_ticker_data.loc[x, 'reddit_sentiment'] = reddit_sentiment
    
sp500_ticker_data

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alp

time    open    low  close  volume  rsi  obv  \
ATVI  2021-02-26 15:45:00-05:00   96.28  95.53  95.53   51708  NaN  NaN   
AMD   2021-02-26 15:45:00-05:00   84.94  84.49  84.51  258826  NaN  NaN   
AES   2021-02-26 15:45:00-05:00   26.69  26.55  26.57   57512  NaN  NaN   
AFL   2021-02-26 15:45:00-05:00  48.215  47.87  47.88   82604  NaN  NaN   
AKAM  2021-02-26 15:45:00-05:00   94.69  94.42  94.43   40095  NaN  NaN   
...                         ...     ...    ...    ...     ...  ...  ...   
WY    2021-02-26 15:45:00-05:00   34.03  33.88  33.88   78528  NaN  NaN   
WMB   2021-02-26 15:45:00-05:00   22.97  22.82  22.82  166772  NaN  NaN   
XEL   2021-02-26 15:45:00-05:00  58.945  58.58  58.58   66101  NaN  NaN   
XRX   2021-02-26 15:45:00-05:00   25.77  25.49  25.49   62863  NaN  NaN   
ZION  2021-02-26 15:45:00-05:00   53.68  53.19  53.22   13042  NaN  NaN   

     volatility twitter_sentiment reddit_sentiment  
ATVI        NaN               NaN              NaN  
AMD         NaN               NaN              NaN  
AES         NaN               NaN              NaN  
AFL         NaN               NaN              NaN  
AKAM        NaN               NaN              NaN  
...         ...               ...              ...  
WY          NaN               NaN              NaN  
WMB         NaN               NaN              NaN  
XEL         NaN               NaN              NaN  
XRX         NaN               NaN              NaN  
ZION        NaN               NaN              NaN  

[230 rows x 10 columns]

In [32]:
############################################################
""" 

    Adds up scores of each indicator and returns a sorted list with Top 5 stock tickers
    
"""
############################################################

indicators_df = pd.DataFrame(sp500_ticker_data)

# adds up all indicator column scores and calculates the total_indi_score
for rows in range(len(indicators_df)):
    indicators_df[rows, 'total_trading_score'] = float(indicators_df[rows, 'rsi'] + indicators_df[rows, 'obv'] + indicators_df[rows, 'volatility'] + indicators_df[rows, 'twitter_sentiment'] + indicators_df[rows, 'reddit_sentiment'])
    
# sorts indidcator_df by total_indie_score
indicators_df.sort_by(by='total_trading_score', ascending=True)

# saves top 5 tickers into top5_stocks_df["tickers"]
top5_stocks_df = indicators_df[:4]

# return top 5 stock recommendations
top5_stocks_df

KeyError: (0, 'rsi')